In [1]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/anaconda/envs/azureml_py310_sdkv2/bin/python
3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]
sys.version_info(major=3, minor=10, micro=11, releaselevel='final', serial=0)


In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import AmlCompute
import os
from azure.ai.ml.entities import Environment
from azure.ai.ml import command, Input
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy
from azure.identity import DefaultAzureCredential

In [3]:
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


# Intro

Para el tuneo de hiperparámetros para un problema de clasificación o regresión, se crea el modelo utilizando librerias de ML como ser pytorch, scikit-learn, tensorflow, keras,etc. Luego, se configura los hiperparámetros propios del modelo a ser tuneados. 
En este caso, utilizaremos una máquina de vectores de soporte (SVM). Es un modelo de aprendizaje automático muy potente y versátil, capaz de realizar clasificación, regresión e incluso detección de valores atípicos lineales o no lineales. 

Para el dataset de `weatherAUS_ML.csv` tunearemos 2 hiperparámetros: 
- `C` 
- `kernel` 

Nota: El dataset fue previamente limpiado y esta almacenado en el siguiente repositorio: 

https://github.com/sharonmaygua/rain_prediction

# Crear un cluster

Se utiliza el nombre de `cpu-cluster`, la familia `STANDARD_DS3_V2` y `4` instancias dedicadas.

In [4]:
cpu_compute_target = "cpu-cluster"

try:
    # let's see if the compute target already exists
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
except Exception:
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        type="amlcompute",
        size="STANDARD_DS3_V2",
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=180,
        tier="Dedicated",
    )

    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster).result()

# Crear un entorno personalizado

Esto nos va a permitir que las instancias de cluster no tengan problema ejecutando el código de `src/train.py`. El archivo `./env/conda.yaml` contiene todas las dependencias necesarias que se instalaran en las instancias.

In [30]:
env_name = 'sklearn-env'
job_env = Environment(
    name=env_name,
    description="sklearn 0.24.2",
    conda_file='./env/conda.yaml',
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)
job_env = ml_client.environments.create_or_update(job_env)


# Ejecutar el código en el cluster

Este comando nos va a permitir correr el script ./src/train.py en la instancia. Se hace referencia al dataset que se va a usar que está previamente cargado en el Datastorage de azure. También, como es solamente una instancia que va a correr, se le pasan los parámetros necesarios utilizados por el script, que son min_samples_split y criterion.

In [46]:
job = command(
    code="./src",
    command="python train.py --dataset_path ${{inputs.dataset_path}} --C ${{inputs.C}} --kernel ${{inputs.kernel}}",
    environment=f"{job_env.name}:{job_env.version}",
    experiment_name='rain-prediction-exp-svm-hyp',
    display_name="rain-prediction-exp-svm-hyp",
    inputs={
        "dataset_path": Input(
            type="uri_file",
            path='https://raw.githubusercontent.com/sharonmaygua/rain_prediction/main/weatherAUS_ML.csv',
        ),
        "C"    : 0.1,
        "kernel": 'rbf',   
    },
    compute=cpu_compute_target,
)


In [47]:
returned_job = ml_client.jobs.create_or_update(job)

Uploading src (0.0 MBs): 100%|██████████| 2842/2842 [00:00<00:00, 40671.38it/s]




In [48]:
ml_client.jobs.stream(returned_job.name)

RunId: quirky_gas_y08nzc6vpp
Web View: https://ml.azure.com/runs/quirky_gas_y08nzc6vpp?wsid=/subscriptions/3deaa453-5a6c-4bcd-85f1-1645c3ccd539/resourcegroups/diplomado_ucb/workspaces/rain_prediction_aus

Execution Summary
RunId: quirky_gas_y08nzc6vpp
Web View: https://ml.azure.com/runs/quirky_gas_y08nzc6vpp?wsid=/subscriptions/3deaa453-5a6c-4bcd-85f1-1645c3ccd539/resourcegroups/diplomado_ucb/workspaces/rain_prediction_aus



# Tuneo de hiperparámetros

En este caso, se reusa el job creado anteriormente pero como una función que configura el espacio de estados de búsqueda para los dos hiperparámetros. Luego se utiliza el método `sweep` para configurar cómo se desea hacer la búsqueda.

In [49]:
job_for_sweep = job(
    C=Choice(values=[0.01,1,100]),
    kernel=Choice(values=['rbf','sigmoid']),
    
)

sweep_job = job_for_sweep.sweep(
    compute=cpu_compute_target,
    sampling_algorithm="random",
    primary_metric="F1 Score",
    goal="Maximize",
    max_total_trials=12,
    max_concurrent_trials=4,
)

returned_sweep_job = ml_client.create_or_update(sweep_job)
ml_client.jobs.stream(returned_sweep_job.name)

RunId: dynamic_pillow_x5j6t9c015
Web View: https://ml.azure.com/runs/dynamic_pillow_x5j6t9c015?wsid=/subscriptions/3deaa453-5a6c-4bcd-85f1-1645c3ccd539/resourcegroups/diplomado_ucb/workspaces/rain_prediction_aus

Streaming azureml-logs/hyperdrive.txt

[2023-10-29T23:39:06.095167][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-10-29T23:39:06.6395089Z][SCHEDULER][INFO]Scheduling job, id='dynamic_pillow_x5j6t9c015_0' 
[2023-10-29T23:39:06.7697955Z][SCHEDULER][INFO]Scheduling job, id='dynamic_pillow_x5j6t9c015_1' 
[2023-10-29T23:39:06.8922675Z][SCHEDULER][INFO]Scheduling job, id='dynamic_pillow_x5j6t9c015_2' 
[2023-10-29T23:39:06.9930495Z][SCHEDULER][INFO]Successfully scheduled a job. Id='dynamic_pillow_x5j6t9c015_0' 
[2023-10-29T23:39:07.0519509Z][SCHEDULER][INFO]Scheduling job, id='dynamic_pillow_x5j6t9c015_3' 
[2023-10-29T23:39:06.912245][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-10-29T23:39:0

In [22]:
import shutil
shutil.make_archive('.', 'zip', './aldunatelipa')

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/aldunatelipac1/code/Users/aldunatelipac/..zip'